In [1]:
import cv2
import os
from tensorflow import keras
from keras.models import load_model
import numpy as np
from pygame import mixer
from playsound import playsound
import time

pygame 2.1.2 (SDL 2.0.18, Python 3.8.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
model = load_model('../models/CNNBasic.h5')

In [3]:
faceCascade = cv2.CascadeClassifier(os.path.dirname(cv2.__file__) + "/data/haarcascade_frontalface_default.xml")
eye_cascade_main = cv2.CascadeClassifier(os.path.dirname(cv2.__file__) + "/data/haarcascade_eye.xml")
#eye_cascade = cv2.CascadeClassifier(os.path.dirname(cv2.__file__) + "/data/haarcascade_eye_tree_eyeglasses.xml")
eye_cascade_backup = cv2.CascadeClassifier(os.path.dirname(cv2.__file__) + "/data/haarcascade_righteye_2splits.xml")

img_size = 128

In [4]:
def preprocessing(frame, gray, score):
    eye_cascade = eye_cascade_main
    
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = frame[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.2, minNeighbors=5)
    
    #change eye_cascade if couldn't detcet eye with current eye_cascade
    if(len(eyes)==0):
        eye_cascade = eye_cascade_backup
        eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.2, minNeighbors=5)
        
    for (ex,ey,ew,eh) in eyes: #green
        eye = roi_color[ey:ey+eh, ex:ex+ew]
        eye = cv2.resize(eye, (img_size, img_size))

        cv2.rectangle(roi_color,(ex, ey),(ex+ew, ey+eh),(0,255,0), 2)
        eye = np.expand_dims(eye, axis=0)
        eye= eye / 255
        
        score = prediction(eye, score)
        break
        
    return score

In [5]:
def prediction(eye, score):
    prediction = model.predict(eye)
    if(prediction[0][0]> 0.5):
        cv2.putText(frame, 'Open', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)
        return False
    else:
        cv2.putText(frame, 'Close', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 2) 
        return True

In [6]:
from datetime import datetime

cap = cv2.VideoCapture(0)
score = 0
frame_rate = 10
prev = datetime.now()

while True:
    cur_time = datetime.now()
    ret, frame = cap.read()
    time_elapsed = (cur_time - prev).total_seconds() * 1000
    # add GaussianBlur to remove noises
    frame = cv2.GaussianBlur(frame, (1, 1), 0)
    height,width = frame.shape[:2] 
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, minNeighbors=5,scaleFactor=1.1)
    cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )
    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)
        score = preprocessing(frame, gray, score)
        if not score:
            prev = datetime.now()
        if time_elapsed > 650 and score:
            print("sleep")
            prev = datetime.now()
            score = False
            playsound('./alarm.wav')
        break
        
    cv2.imshow('h', frame)
    
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

sleep
sleep
sleep
sleep
sleep
